# Ipynb file for Case 1 Nguyen road network

# Inputs change by case
Set input variables for each cases. \
*For details of each input, please check the paper.

In [1]:
#Maximum number of continuous work zones per day: M. 
 #Case 1-a:100000(a number almost infinity). Case 1-b:20. Case 1-c:5.
maxmachinenum=20

#User cost weight. 
 #Case 1-a,1-b,1-c:1. Case 1-bwf=0:1. Case 1-bU=0:0.
ucrate=1

#Work zone cost weight
 #Case 1-a,1-b,1-c:1. Case 1-bwf=0:0. Case 1-bU=0:1.
wcrate=1

#Probability each section of the road network require repair
repprob=0.05

# Step 0: Preparation
Import modules

In [2]:
import numpy as np
import time
import gurobipy as grb
from itertools import product
import itertools
import numpy as np
import random

Fixed inputs for Nguyen network

In [3]:
#Set of the nodes
nodes=np.sort([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13])


#Set of the links
links_data_tuple=((1,5),(1,12),(4,5),(4,9),(5,6),(5,9),(6,7),(6,10),(7,8),(7,11),(8,2),(9,10),(9,13),(10,11),(11,2),(11,3),(12,6),(12,8),(13,3))

#Ratio of the traffic demand to the original data
odsize=0.8

#Traffic demand per day in the entire network
odmat=[[1,2,400*odsize],[1,3,800*odsize],[4,2,600*odsize],[4,3,200*odsize]]

#Traffic capacity of each link
capacity_data=[800,400,200,800,350,400,800,250,250,300,550,550,600,700,500,300,200,400,600]

#Mile of each link
cost_data_array=[7,9,9,12,3,9,5,13,5,9,9,10,9,6,9,8,7,14,11]

#Number of days in the plan
timespan=365

#Fixed work zone cost
mcucost=500

#Variable work zone cost
wzucost=100

#Traffic capacity ratio in the work zone
Brate=0.5

#Detour route cost
dummy_cost=300


In [4]:
#Modification of type of the inputs
links_data_array = np.array(links_data_tuple)

nodes = np.sort(np.unique(links_data_array.flatten()))

cost_dict=dict(zip(links_data_tuple,cost_data_array))

capacity_dict=dict(zip(links_data_tuple,capacity_data))

od_dict={(1,2):400*odsize,(1,3):800*odsize,(4,2):600*odsize,(4,3):200*odsize}

Coding of supportive functions

In [5]:
def convert_to_base(decimal_number, base):
    DIGITS = '0123456789abcdef'
    remainder_stack = []

    while decimal_number > 0:
        remainder = decimal_number % base
        remainder_stack.append(remainder)
        decimal_number = decimal_number // base

    new_digits = []
    while remainder_stack:
        new_digits.append(DIGITS[remainder_stack.pop()])

    return ''.join(new_digits)
def de_to_base(de,base,num):
    de2=convert_to_base(de, base)
    otpt=np.zeros(num, dtype=int)
    i=-1
    for x in de2:
        otpt[i]=otpt[i]+int(de2[i])
        i=i-1
    return otpt

def base_to_de(X,n):
    out = 0
    for i in range(len(X)):
        out += int(X[-i-1])*(n**(i))
    return out

def list_to_tuple(_list):
    t = ()
    for e in _list:
        if isinstance(e,list):
            t += (list_to_tuple(e),)
        else:
            t += (e,)
    return t

def odmatrix(nodes,node_od,sizerate): 
    odmat=np.zeros(3,'int')
    for i in nodes: 
        for j in nodes: 
            if node_od[i][j]>0: 
                odtmp=np.zeros(3,'int')
                odtmp[0]=i 
                odtmp[1]=j 
                odtmp[2]=node_od[i][j]*sizerate
                odmat=np.vstack([odmat,odtmp])
    odmat=np.delete(odmat,0,0)
    return odmat 

def makeod_dict(nodes,node_od,sizerate):
    odpairdict=()
    odarray=[]
    for i in node_od:
        for j in node_od[i]:
            odpairdict=odpairdict+((i,j),)
            odarray.append(node_od[i][j])
    od_dict=dict(zip(odpairdict,odarray))
    return od_dict

def noise(prob):
    return 1 if random.random() <= prob else 0

def random_rep_id(linklength,repprob,seed=21,secnumperlink=1609/100):
    random.seed(seed)
    linksize=round(linklength*secnumperlink)
    randomid_link=[]
    for i in range(linksize):
            randomid_link.append(noise(repprob))
    return randomid_link

# Step 1: Lower level: Section-unit link-level Pareto frontier


In [6]:
#Function for calculating optimal 1 day work zone schedule with constraints of maximum number of continuous work zones per day.
def linkrepplans(repid,machinecost=1000, workzonecost=100,maxworkzonelength=10,maxmachinenum=100):
    repplangrbm=()
    with grb.Env(empty=True) as env:
        env.setParam('OutputFlag', 0)
        env.start()
        with grb.Model(env=env) as repplangrbm:
            linksize=len(repid)
            link_machineid={}
            if maxmachinenum>sum(repid):
                maxmachinenum=sum(repid)
            link_machineid=repplangrbm.addVars(linksize,vtype=grb.GRB.INTEGER,lb=0,ub=maxmachinenum)
            workzone01={}

            workzone01=repplangrbm.addVars(linksize,vtype=grb.GRB.BINARY)

            repplangrbm.update()

            machineidincrease=repplangrbm.addConstrs( link_machineid[i+1]>= link_machineid[i] for i in range(linksize-1))
            machineworkzoneconst=repplangrbm.addConstrs((workzone01[i+1]-workzone01[i])*(link_machineid[i+1]-link_machineid[i]-0.5)>=0 for i in range(linksize-1))
            workzonelengthconst=repplangrbm.addConstrs(repid[i+maxworkzonelength]*(link_machineid[i+maxworkzonelength]-link_machineid[i]-1)>=0   for i in range(linksize-maxworkzonelength))
            workzone01const=repplangrbm.addConstrs( repid[i]<= workzone01[i] for i in range(linksize))
            machine01const=repplangrbm.addConstrs( workzone01[i]<= link_machineid[i] for i in range(linksize))

            repplangrbm.setObjective(grb.quicksum(workzone01[i] for i in range(linksize))*workzonecost,grb.GRB.MINIMIZE)
            repplangrbm.update()
            repplangrbm.optimize()
            repplangrbm.status
            
            link_machineidopt=[]
            workzonenum=0
            try:
                for i in range(linksize):
                    link_machineidopt.append(link_machineid[i].X)
                    workzonenum=workzonenum+workzone01[i].X
                repidans=[workzone01[i].X for i in range(linksize)]
            except:
                workzonenum=np.inf
                repidans=np.inf
            return workzonenum, repidans
            
        


In [7]:
#Function for calculate Pareto frontier of each link.
#   For each link, first randomly generate repair pattern,
#   then calculate the optimal 1 day work zone schedule while decreasing the maximum number of continuous work zones per day
def alllinksparato(cost_data,maxmachineperday=100,maxwznum=10,repprob=0.2,seedstart=0):
    linksnum=len(cost_data)
    alllinksparatocost=np.zeros((linksnum,maxmachineperday))
    allrepid=[]
    allworkzoneid=[[0 for i in range(maxmachineperday)] for j in range(linksnum)]
    for linkid in range(linksnum):
        linkrepid=random_rep_id(cost_data[linkid],repprob,seed=linkid+seedstart)
        allrepid.append(linkrepid)
        for machinenum in range(maxmachineperday):
            try:
                alllinksparatocost[linkid][machinenum], allworkzoneid[linkid][machinenum]=linkrepplans(linkrepid,machinecost=1000, workzonecost=100,maxworkzonelength=maxwznum,maxmachinenum=machinenum)
            except:
                alllinksparatocost[linkid][machinenum], allworkzoneid[linkid][machinenum]=np.inf,np.inf
    return  alllinksparatocost, allrepid, allworkzoneid


In [8]:
#Calculation of Pareto frontier of each link
wzformn, allrepidmn, allworkzoneidmn=alllinksparato(cost_data_array,100,10,repprob)


In [9]:
#Pareto frontiers into boardlist for the next step
maxmachineperday=100
boardlist=[]
for linkid in range(len(cost_data_array)):
    boardlinklist=[]
    tmpwznm=1000000
    for j in range(maxmachineperday):
            if wzformn[linkid][j]<=100000 and wzformn[linkid][j]<tmpwznm:
                boardlinklist.append([j,round(wzformn[linkid][j])])
                tmpwznm=wzformn[linkid][j]
    boardlist.append(boardlinklist)    

# Step 2: Upper level: Link-unit network-level optimal work zone assignment

Define Gurobi Model for upper level

In [10]:
timerange=range(timespan)
grbm=()
grbm=grb.Model()
x={}
y={}
FlowRsv={}
Cap={}
close={}
CloseCons={}
RepairOnce={}
cost_origin_dict=dict(zip(links_data_tuple,cost_data_array))
flowtype_list=range(len(odmat))
links_data2_array=links_data_array#
links_data2_tuple=links_data_tuple#
cost_data_array2_array=cost_data_array
capacity_data2_array=capacity_dict
detour_tuple=()
detour_list=[[0,0],]
detour_cost_array=[]
for f in flowtype_list:
    detour_tuple=detour_tuple+list_to_tuple([[odmat[f][0],odmat[f][1]]])
    detour_list=np.row_stack((detour_list, odmat[f][:2]))
    detour_cost_array=np.append( detour_cost_array,3000)
detour_list=np.delete(detour_list,0,0)
detour_dict=dict(zip(detour_tuple,detour_cost_array))
outflow_dict=dict(zip([(i,f) for (i,f) in product(nodes,flowtype_list)],np.zeros(len(nodes)*len(flowtype_list))))
count=0
for f in flowtype_list:
    onode=odmat[f][0]
    dnode=odmat[f][1]
    flow=odmat[f][2]
    outflow_dict[(onode,f)]=-flow
    outflow_dict[(dnode,f)]=flow
ynwlist=list(itertools.product(links_data_tuple,np.array(flowtype_list),np.array(timerange)))
ydetourlist=list(itertools.product(detour_tuple,np.array(timerange)))
ynw=grbm.addVars(ynwlist,lb=0)
ydet=grbm.addVars(ydetourlist,lb=0)
xlist=list(itertools.product(links_data_tuple,np.array(timerange)))
 

x01num=grbm.addVars(xlist, vtype="B")

xnum=grbm.addVars(xlist, vtype="I", lb=0,ub=max(maxmachinenum,100))

xtlist=list(links_data_tuple)
xtnum=grbm.addVars(xtlist, vtype="I", lb=0,ub=max(maxmachinenum,100))

wznum=grbm.addVars(xtlist, vtype="I", lb=0,ub=max(maxmachinenum*10,1000))

# Define detour routes on the network
detour_indicator=dict(zip([(i,f) for (i,f) in product(nodes,flowtype_list)],np.zeros(len(nodes)*len(flowtype_list))))
count=0
for f in flowtype_list:
    for i in nodes:
        if detour_list[f][0]==i:
            detour_indicator[(i,f)]=1
        elif detour_list[f][1]==i:
            detour_indicator[(i,f)]=-1

#Decide od nodes for each traffic demands
all_node_link_start_list=[]
all_node_link_end_list=[]
for n in nodes:
    node_link_start_list=[]
    node_link_end_list=[]
    for (i,j) in links_data_array:
        if i==n:
            node_link_start_list.append([i,j])
        elif j==n:
            node_link_end_list.append([i,j])
    all_node_link_start_list.append(list(node_link_start_list))
    all_node_link_end_list.append(list(node_link_end_list))

all_node_detour_start_list=[]
all_node_detour_end_list=[]
for n in nodes:
    node_detour_start_list=[]
    node_detour_end_list=[]
    for (i,j) in detour_list:
        if i==n:
            node_detour_start_list.append([i,j])
        elif j==n:
            node_detour_end_list.append([i,j])
    all_node_detour_start_list.append(list(node_detour_start_list))
    all_node_detour_end_list.append(list(node_detour_end_list))

#Constraints
FlowRsv=grbm.addConstrs(grb.quicksum(ynw[(n,m),f,t]for (n,m) in all_node_link_start_list[i])
+ydet[(detour_list[f][0],detour_list[f][1]),t]*detour_indicator[(i+1,f)] 
- grb.quicksum(ynw[(n,m),f,t]for (n,m) in all_node_link_end_list[i]) 
== -outflow_dict[nodes[i],f] for (f,i,t) in product(flowtype_list, range(len(nodes)),timerange))   
x01consts=grbm.addConstrs(x01num[(i,j),t]*maxmachinenum>=xnum[(i,j),t] for ((i,j),t) in product(links_data_array,timerange))
Cap=grbm.addConstrs(grb.quicksum(ynw[(i,j),f,t] for f in flowtype_list) <= capacity_dict[i,j]*(1-Brate*x01num[(i,j),t]) for ((i,j),t) in product(links_data_array,timerange))
Xnumconst=grbm.addConstrs(xtnum[i,j]==grb.quicksum(xnum[(i,j),t] for t in timerange)for(i,j) in links_data_array  )
mnumconst=grbm.addConstrs(grb.quicksum(xnum[(i,j),t] for (i,j) in links_data_array)<=maxmachinenum for t in timerange )

#Add Pareto frontier into the upper level problem
xtnumind={}
for i in range(len(cost_data_array )):
    (m,n)=links_data_tuple[i]
    grbm.addConstr(xtnum[m,n]>=boardlist[i][0][0])
    grbm.addConstr(xtnum[m,n]<=boardlist[i][-1][0])
    xtnumind[i]=grbm.addVars(len(boardlist[i]), vtype="B")
    grbm.addConstr(grb.quicksum(xtnumind[i][j] for j in range(len(boardlist[i])))==1)
    grbm.addConstr(grb.quicksum(xtnumind[i][j]*boardlist[i][j][1] for j in range(len(boardlist[i])))==wznum[m,n])
    grbm.addConstr(grb.quicksum(xtnumind[i][j]*boardlist[i][j][0] for j in range(len(boardlist[i])))==xtnum[m,n])
    
#Set of objective function
grbm.setObjective(
ucrate*grb.quicksum(cost_origin_dict[n,m]*ynw[(n,m),f,t] for((n,m),f,t) in product(links_data_array,flowtype_list,timerange))
+ucrate*grb.quicksum(detour_dict[n,m]*ydet[(n,m),t] for((n,m),t) in product(detour_list,timerange))
+wcrate*grb.quicksum(xtnum[n,m]*mcucost
+wznum[n,m]*wzucost  
for (n,m) in links_data_array)
)    

grbm.update()
grbm.setParam('MIPGAP',0.01)
grbm.setParam('MIPfocus',3)
grbm.optimize()
grbm.status

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-10
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 3
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i9-9980XE CPU @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 18 physical cores, 36 logical processors, using up to 18 threads

Optimize a model with 2844 rows, 3626 columns and 10169 nonzeros
Model fingerprint: 0xa7b24f0d
Variable types: 2400 continuous, 1226 integer (618 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [3e+00, 3e+03]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 8e+02]
Found heuristic solution: objective 1.440771e+08
Presolve removed 1275 rows and 1577 columns
Presolve time: 0.01s
Presolved: 1569 rows, 2049 columns, 6793 nonzeros
Variable types: 900 continuous, 1149 integer (599 binary)
Found heuristic solution: objective 1.440760e+08

2

In [11]:
sum(xtnum[n,m].X for (n,m) in links_data_array)

107.0

In [12]:
sum(wznum[n,m].X for (n,m) in links_data_array)

217.0

In [13]:
sum(xtnum[n,m].X*500+wznum[n,m].X*100  for (n,m) in links_data_array)

75200.0

In [14]:
sum(cost_origin_dict[n,m]*ynw[(n,m),f,t].X for((n,m),f,t) in product(links_data_array,flowtype_list,timerange))+sum(detour_dict[n,m]*ydet[(n,m),t].X for((n,m),t) in product(detour_list,timerange))+sum(xtnum[n,m].X*500+wznum[n,m].X*100  for (n,m) in links_data_array)

2958985.0

In [15]:
otptx={(i,j,t):xnum[(i,j),t].X for ((i,j),t) in product(links_data_array,timerange)}
print(otptx)

{(1, 5, 0): 2.0, (1, 5, 1): 0.0, (1, 5, 2): 0.0, (1, 5, 3): 0.0, (1, 5, 4): 0.0, (1, 5, 5): 0.0, (1, 5, 6): 0.0, (1, 5, 7): 0.0, (1, 5, 8): 0.0, (1, 5, 9): 0.0, (1, 5, 10): 0.0, (1, 5, 11): 0.0, (1, 5, 12): 0.0, (1, 5, 13): 0.0, (1, 5, 14): 0.0, (1, 5, 15): 0.0, (1, 5, 16): 0.0, (1, 5, 17): 0.0, (1, 5, 18): 0.0, (1, 5, 19): 0.0, (1, 5, 20): 0.0, (1, 5, 21): 0.0, (1, 5, 22): 0.0, (1, 5, 23): 0.0, (1, 5, 24): 0.0, (1, 5, 25): 0.0, (1, 5, 26): 0.0, (1, 5, 27): 0.0, (1, 5, 28): 0.0, (1, 5, 29): 0.0, (1, 12, 0): 0.0, (1, 12, 1): 0.0, (1, 12, 2): 0.0, (1, 12, 3): 0.0, (1, 12, 4): 0.0, (1, 12, 5): 0.0, (1, 12, 6): 0.0, (1, 12, 7): 0.0, (1, 12, 8): 0.0, (1, 12, 9): 0.0, (1, 12, 10): 0.0, (1, 12, 11): 0.0, (1, 12, 12): 0.0, (1, 12, 13): 0.0, (1, 12, 14): 0.0, (1, 12, 15): 0.0, (1, 12, 16): 0.0, (1, 12, 17): 0.0, (1, 12, 18): 0.0, (1, 12, 19): 0.0, (1, 12, 20): 0.0, (1, 12, 21): 0.0, (1, 12, 22): 0.0, (1, 12, 23): 0.0, (1, 12, 24): 0.0, (1, 12, 25): 0.0, (1, 12, 26): 0.0, (1, 12, 27): 9.0, (1, 1

with rhot 222sec 1.98%